# Modèle V1 de voiture autonome sur DonkeyCarSimulator

In [ ]:
import os
from google.colab import files
from google.colab import drive
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from time import time

In [ ]:
TIME=str(time())
MODEL_NAME = "DCModelV1-"+TIME

STORAGE_ROOT_DIR = "drive/My Drive/ColabStorage"
PROJECT_PATH = "DonkeyCar/Simulator"
PROJECT_ROOT_DIR = os.path.join(STORAGE_ROOT_DIR,PROJECT_PATH)
DATASET_NAME = "3_controller_generated_track_0_48000.zip"
DRIVE_DATASET_PATH = os.path.join(PROJECT_ROOT_DIR, "Dataset", DATASET_NAME)
DATASET_PATH = DATASET_NAME[:-4]
TENSORLOG_PATH = os.path.join(STORAGE_ROOT_DIR, PROJECT_PATH, "__TensorLog__")

BATCH_SIZE = 32

SAVE_PATH = os.path.join(STORAGE_ROOT_DIR, PROJECT_PATH, "Model")
IMAGE_SHAPE = (120,160, 1)
drive.mount('/content/drive', force_remount=True)
print(DRIVE_DATASET_PATH)
print("Ok.")

Mounted at /content/drive
drive/My Drive/ColabStorage/DonkeyCar/Simulator/Dataset/3_controller_generated_track_0_48000.zip
Ok.


## 1. Préparer la dataset

In [ ]:
!rm -Rf 3_controller_generated_track_0_48000 
!unzip -q "/content/drive/My Drive/ColabStorage/DonkeyCar/Simulator/Dataset/3_controller_generated_track_0_48000.zip" -d ./
!ls

3_controller_generated_track_0_48000  drive  sample_data


In [ ]:
!ls 3_controller_generated_track_0_48000

images	label.csv


In [ ]:
# Obetnir les valeurs du fichier contenant les labels
raw_data = pd.read_csv(os.path.join(DATASET_PATH, "label.csv"))

# change la chemin de fichier
def change_path(path):
  return os.path.join(DATASET_PATH, path)
raw_data['path'] = raw_data['path'].map(change_path)
print(raw_data)

                                                    path  angle  throttle
0      3_controller_generated_track_0_48000/images/20...    1.0       0.0
1      3_controller_generated_track_0_48000/images/20...    1.0       0.0
2      3_controller_generated_track_0_48000/images/20...    1.0       0.0
3      3_controller_generated_track_0_48000/images/20...    1.0       0.0
4      3_controller_generated_track_0_48000/images/20...    1.0       0.0
...                                                  ...    ...       ...
47996  3_controller_generated_track_0_48000/images/49...    1.0       0.0
47997  3_controller_generated_track_0_48000/images/49...    1.0       0.0
47998  3_controller_generated_track_0_48000/images/49...    1.0       0.0
47999  3_controller_generated_track_0_48000/images/49...    1.0       0.0
48000  3_controller_generated_track_0_48000/images/50...    1.0       0.0

[48001 rows x 3 columns]


### Split en 3 jeux : Train, Test et Validation

In [ ]:
train_and_test_set, validation_set = train_test_split(raw_data,
                                             test_size = 0.15,
                                             shuffle = True)
train_set, test_set = train_test_split(train_and_test_set,
                                             test_size = 0.20,
                                             shuffle = True)
NBR_ROW_TRAIN_SET = train_set.shape[0]
NBR_ROW_TEST_SET = test_set.shape[0]
NBR_ROW_VALIDATION_SET = validation_set.shape[0]
print(train_set)
print(test_set)
print(validation_set)

                                                    path     angle  throttle
46524  3_controller_generated_track_0_48000/images/48...  1.000000  0.126342
39821  3_controller_generated_track_0_48000/images/41...  1.000000  0.000000
32264  3_controller_generated_track_0_48000/images/34...  1.000000 -0.290916
31335  3_controller_generated_track_0_48000/images/33...  1.000000  0.000000
28557  3_controller_generated_track_0_48000/images/30...  0.000000  1.000000
...                                                  ...       ...       ...
21952  3_controller_generated_track_0_48000/images/23...  0.151623  1.000000
34204  3_controller_generated_track_0_48000/images/36... -0.759469  1.000000
46821  3_controller_generated_track_0_48000/images/48...  0.000000  1.000000
18273  3_controller_generated_track_0_48000/images/20... -0.468224  1.000000
33715  3_controller_generated_track_0_48000/images/35...  1.000000  0.000000

[32640 rows x 3 columns]
                                                  

### Traitements avec TensorData

In [ ]:
# Mettre dans des tensors
train_tensor = tf.data.Dataset.from_tensor_slices(({"input" : train_set['path']}, {"angle" : train_set['angle'], "throttle" : train_set['throttle']}))
test_tensor = tf.data.Dataset.from_tensor_slices(({"input" : test_set['path']}, {"angle" : test_set['angle'], "throttle" : test_set['throttle']}))
validation_tensor = tf.data.Dataset.from_tensor_slices(({"input" : validation_set['path']}, {"angle" : validation_set['angle'], "throttle" : validation_set['throttle']}))

# Definir les fonctions de chargement des images et de mapping
def load_and_preprocess_image(path):
  file_content = tf.io.read_file(path['input'])
  tricolors_img = tf.cast(tf.image.decode_jpeg(file_content, channels=3), dtype=tf.float32)
  gray_img = tf.image.rgb_to_grayscale(tricolors_img)
  # FIX change / 255.0 by / 255.0  -  0.5, to have a range between -0.5 et 0.5 (instead of 0 and 1)
  normalized_img = gray_img / 255.
  print(normalized_img)
  return {"input": tf.reshape(normalized_img, IMAGE_SHAPE)}

def load_map_function(path, d):
  return load_and_preprocess_image(path), d

# Appliquer le mapping aux tensors
train_tensor_normalized = train_tensor.map(load_map_function, num_parallel_calls=3)
test_tensor_normalized = test_tensor.map(load_map_function, num_parallel_calls=3)
validation_tensor_normalized = validation_tensor.map(load_map_function, num_parallel_calls=3)

Tensor("truediv:0", shape=(None, None, 1), dtype=float32)
Tensor("truediv:0", shape=(None, None, 1), dtype=float32)
Tensor("truediv:0", shape=(None, None, 1), dtype=float32)


In [ ]:
# On mélange les datasets (on fixe un nombre d'exemple tiré au sort, ici 20 000 exemples sur les 30 000)
# On demande un prechargement à l'avance de toujours 3 exemples
train_dataset = train_tensor_normalized.shuffle(20000).batch(BATCH_SIZE).prefetch(2)
test_dataset = test_tensor_normalized.shuffle(8000).batch(BATCH_SIZE).prefetch(2)
validation_dataset = validation_tensor_normalized.shuffle(8000).batch(BATCH_SIZE).prefetch(2)

## 2. Le modèle

In [ ]:
LR = 0.0001

In [ ]:
class DCModel(keras.Model):
  def __init__(self, name=None):
    super(DCModel, self).__init__(name=name)
    #self.input_layer = keras.layers.Input(shape=IMAGE_SHAPE, name='input')
    self.cnn_1 = keras.layers.Conv2D(32, (5, 5), strides=(2, 2) , kernel_initializer='he_uniform', activation='relu', name='input2')
    self.cnn_2 = keras.layers.Conv2D(64, (3, 3), strides=(2, 2) , kernel_initializer='he_uniform', activation='relu')
    
    self.max_1 = keras.layers.MaxPooling2D((2, 2))
    
    self.cnn_3 = keras.layers.Conv2D(128, (3, 3), strides=(1, 1) , kernel_initializer='he_uniform', activation='relu')
    self.drop_1 = keras.layers.Dropout(0.4)

    self.flat = keras.layers.Flatten()

    self.dense_1 = keras.layers.Dense(2000, kernel_initializer='he_uniform', activation='elu')
    self.drop_2 = keras.layers.Dropout(0.4)
    self.dense_2 = keras.layers.Dense(100, kernel_initializer='he_uniform', activation='elu')
    self.dense_3 = keras.layers.Dense(50, kernel_initializer='he_uniform', activation='elu')

    self.output_layer_1 = keras.layers.Dense(1, activation='linear', name='angle')
    self.output_layer_2 = keras.layers.Dense(1, activation='linear', name='throttle')

  @tf.function
  def call(self, inputs):
    l = self.cnn_1(inputs['input'])
    l = self.cnn_2(l)
    l = self.max_1(l)
    l = self.cnn_3(l)
    l = self.drop_1(l)
    l = self.flat(l)
    l = self.dense_1(l)
    l = self.drop_2(l)
    l = self.dense_2(l)
    l = self.dense_3(l)
    return {'angle' : self.output_layer_1(l), 'throttle' : self.output_layer_2(l)}

model = DCModel(name='DonkeyCarModel')

"""
input_layer = keras.layers.Input(shape=IMAGE_SHAPE, name='input')
l = keras.layers.Conv2D(32, (5, 5), strides=(2, 2) , kernel_initializer='he_uniform', activation='relu')(input_layer)
l = keras.layers.Conv2D(64, (3, 3), strides=(2, 2) , kernel_initializer='he_uniform', activation='relu')(l)
l = keras.layers.MaxPooling2D((2, 2))(l)
l = keras.layers.Conv2D(128, (3, 3), strides=(1, 1) , kernel_initializer='he_uniform', activation='relu')(l)
l = keras.layers.Dropout(0.4)(l)
l = keras.layers.Flatten()(l)
l = keras.layers.Dense(2000, kernel_initializer='he_uniform', activation='elu')(l)
l = keras.layers.Dropout(0.4)(l)
l = keras.layers.Dense(100, kernel_initializer='he_uniform', activation='elu')(l)
l = keras.layers.Dense(50, kernel_initializer='he_uniform', activation='elu')(l)
output_layers = [ keras.layers.Dense(1, activation='tanh', name='angle')(l), keras.layers.Dense(1, activation='tanh', name='throttle')(l) ]

model = keras.Model(inputs=[input_layer], outputs=output_layers)
"""

optimizer = keras.optimizers.Nadam(learning_rate=LR)
# FIX au pire mettre une activation lineaire sur l'output_layers si ça ne converge pas
model.compile(optimizer=optimizer,loss=keras.losses.MSE, metrics=["accuracy"])

## 3. L'entrainement

In [ ]:
#@title Les hyperparamètres de l'entrainement
NBR_EPOQUES = 1 #@param {type:"slider", min:0, max:100, step:1}
NBR_EPOQUES_APRES_EARLY_STOPPING = 4 #@param {type:"slider", min:0, max:100, step:1}

In [ ]:
# Définir mes callback
cb = [
    keras.callbacks.EarlyStopping(patience=NBR_EPOQUES_APRES_EARLY_STOPPING,
                                  restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir=TENSORLOG_PATH)
    ]

# Fit mon modèle
model.fit(train_dataset,
          validation_data=test_dataset,
          epochs=NBR_EPOQUES,
          callbacks=cb)

1020/1020 [==============================] - 76s 59ms/step - loss: 2.7287 - angle_loss: 2.4038 - throttle_loss: 0.3249 - angle_accuracy: 0.6091 - throttle_accuracy: 0.6020 - val_loss: 0.1049 - val_angle_loss: 0.0482 - val_throttle_loss: 0.0568 - val_angle_accuracy: 0.7179 - val_throttle_accuracy: 0.7156


In [ ]:
model.summary()

Model: "DonkeyCarModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input2 (Conv2D)              multiple                  832       
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  73856     
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple               

In [ ]:
#Tensorboad
%load_ext tensorboard
%tensorboard --logdir="drive/My Drive/ColabStorage/DonkeyCar/Simulator/__TensorLog__/"

In [ ]:
#model.save(os.path.join(STORAGE_ROOT_DIR, PROJECT_PATH, "Model", MODEL_NAME), save_format='tf')
model.save(os.path.join(STORAGE_ROOT_DIR, PROJECT_PATH, "Model.h5", MODEL_NAME), save_format='h5')

INFO:tensorflow:Assets written to: drive/My Drive/ColabStorage/DonkeyCar/Simulator/Model/DCModelV1-1612557212.3859317/assets


In [ ]:
performances = model.evaluate(validation_dataset, batch_size=32)
print("Loss Angle :", performances[0],"Loss Throttle :", performances[1],"Acc Angle :", performances[2],"Acc Throttle :", performances[3])

226/226 [==============================] - 4s 6ms/step - loss: 0.0952 - angle_loss: 0.0418 - throttle_loss: 0.0534 - angle_accuracy: 0.7067 - throttle_accuracy: 0.7089
Loss Angle : 0.095182865858078 Loss Throttle : 0.04182043299078941 Acc Angle : 0.05336245149374008 Acc Throttle : 0.7067074179649353
